In [1]:
import os
os.environ['PROJ_LIB'] = '/data/keeling/a/sshu3/anaconda2/share/proj'
from mpl_toolkits.basemap import Basemap, cm, shiftgrid,maskoceans
from netCDF4 import Dataset as NetCDFFile
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy.ma as ma
from scipy.interpolate import griddata
from pyresample import geometry,image, kd_tree
import copy

In [2]:
# Read in model output
# Path
respath='/data/jain1/c/sshu3/SBGC/results/SBGC_regional/'
contemp='Global_1DSBGC.bgc-yearly-3d_2009.nc'
preind='Global_1DSBGC.bgc-yearly-3d_1960.nc'

# Read the 3D files, For ISAM-1D
# 1960
fp=respath+preind
nclu_preind=NetCDFFile(fp,'r')
soclev_preind = nclu_preind.variables['soil_c_lev'][:]
resplev_preind = nclu_preind.variables['resp_lev'][:]
latnc = nclu_preind.variables['lat'][:]
lonnc = nclu_preind.variables['lon'][:]
#lonnc = ((lonnc - 180) % 360) - 180
nclu_preind.close()

# 1960
fp=respath+contemp
nclu_contemp=NetCDFFile(fp,'r')
soclev_contemp = nclu_contemp.variables['soil_c_lev'][:]
resplev_contemp = nclu_contemp.variables['resp_lev'][:]
#lonnc = ((lonnc - 180) % 360) - 180
nclu_contemp.close()

# Calculate the SOC for the first 1m
# Get the topsoil SOC (0-30cm) from 1D model
soc_top_preind = np.sum(soclev_preind[0:5,:,:], 0)
soc_top_contemp = np.sum(soclev_contemp[0:5,:,:], 0)

# topsoil resp
resp_top_preind = np.sum(resplev_preind[0:5,:,:], 0)
resp_top_contemp = np.sum(resplev_contemp[0:5,:,:], 0)

# Get the subsoil SOC (30-100cm) from 1D model
soc_sub_preind = np.sum(soclev_preind[5:7,:,:], 0)+0.3*soclev_preind[7,:,:]
soc_sub_contemp = np.sum(soclev_contemp[5:7,:,:], 0)+0.3*soclev_contemp[7,:,:]

# subsoil resp
resp_sub_preind = np.sum(resplev_preind[5:7,:,:], 0)+0.3*resplev_preind[7,:,:]
resp_sub_contemp = np.sum(resplev_contemp[5:7,:,:], 0)+0.3*resplev_contemp[7,:,:]

# Get the first 1m SOC
soc_preind = soc_top_preind + soc_sub_preind
soc_contemp = soc_top_contemp + soc_sub_contemp
dsoc = soc_contemp - soc_preind

# Path
respath='/data/jain1/c/sshu3/SBGC/results/SBGC_0d/'
contemp='Global_0DSBGC.bgc-yearly-2d_2009.nc'
preind='Global_0DSBGC.bgc-yearly-2d_1960.nc'

# Read the 2D files, For ISAM-0D
# 1960
fp=respath+preind
nclu_preind=NetCDFFile(fp,'r')
soc0d_preind = nclu_preind.variables['soil_c'][:]
resp0d_preind = nclu_preind.variables['resp'][:]
#lonnc = ((lonnc - 180) % 360) - 180
nclu_preind.close()

# 20th century
fp=respath+contemp
nclu_contemp=NetCDFFile(fp,'r')
soc0d_contemp = nclu_contemp.variables['soil_c'][:]
resp0d_contemp = nclu_contemp.variables['resp'][:]
nclu_contemp.close()

# Calculate the SOC for the first 1m
dsoc0d = soc0d_contemp - soc0d_preind

# Obatin permafrost area mask from Brown et al., 1998 [0<code<23 are permafrost]
# Open NCSCD as observation
path='/data/jain1/c/sshu3/SBGC/data/ggd318_permafrost_and_ice_map_circumarctic/permafrost_mask.nc'
#path='/data/jain1/c/sshu3/SBGC/data/NCSCDv2_Circumpolar_netCDF_05deg/ncscd_masked_0.5x0.5.nc'
nclu_pf=NetCDFFile(path,'r')
pfcode = nclu_pf.variables['PFCODE'][:]
latnc_pf = nclu_pf.variables['Lat'][:]
lonnc_pf = nclu_pf.variables['Lon'][:]
nclu_pf.close()
temp = copy.deepcopy(pfcode[:, 0:360])
pfcode[:, 0:360] = pfcode[:, 360:720]
pfcode[:, 360:720] = temp
pfcode=pfcode[::-1,:]
temp = ma.masked_where(pfcode<1, pfcode)
ncscd_mask = ma.masked_where(temp>22, pfcode)

# Get the country mask of Norway (45400), Finland (42100), Sweden (47000) and Russia (46100)
path='/data/jain1/c/sshu3/SBGC/data/C14/C14processing/Ctry_halfdeg.nc'
nclu_ctry=NetCDFFile(path,'r')
ctrycode = nclu_ctry.variables['MASK_Country'][:]
latnc_ctry = nclu_ctry.variables['Lat'][:]
lonnc_ctry = nclu_ctry.variables['Lon'][:]
nclu_pf.close()
norway = ma.masked_where(ctrycode != 45400, ctrycode)
finland = ma.masked_where(ctrycode != 42100, ctrycode)
sweden = ma.masked_where(ctrycode != 47000, ctrycode)
russia = ma.masked_where(ctrycode != 46100, ctrycode)

# Get the BONA(1), BOAS(7) and Europe (11) region mask from transcom region mask data 
# The Europe part is read in to exclude Asian Russia from the whole Russia
path='/data/jain1/c/sshu3/SBGC/data/C14/C14processing/transcom_regions_halfdeg.nc'
nclu_reg = NetCDFFile(path,'r')
regcode = nclu_reg.variables['transcom_regions'][:]
latnc_reg = nclu_reg.variables['lat'][:]
lonnc_reg = nclu_reg.variables['lon'][:]
nclu_reg.close()
bona = ma.masked_where(regcode != 1, regcode)
boas = ma.masked_where(regcode != 7, regcode)
eu = ma.masked_where(regcode != 11, regcode)
eu_russia = ma.masked_where(eu.mask, russia)

# Combine three together, which is the same mask as Dave MCGuire's paper
temp_mask =eu_russia.mask & finland.mask & norway.mask & sweden.mask & bona.mask & boas.mask
temp = copy.deepcopy(temp_mask[:, 0:360])
temp_mask[:, 0:360] = temp_mask[:, 360:720]
temp_mask[:, 360:720] = temp
dave_mask = temp_mask & ncscd_mask.mask

# Mask all regions outside for model outputs
soc_preind_mc = copy.deepcopy(soc_preind)
soc_preind_mc.mask = dave_mask
soc0d_preind_mc = copy.deepcopy(soc0d_preind)
soc0d_preind_mc.mask = dave_mask
soc_contemp_mc = copy.deepcopy(soc_contemp)
soc_contemp_mc.mask = dave_mask
soc0d_contemp_mc = copy.deepcopy(soc0d_contemp)
soc0d_contemp_mc.mask = dave_mask

soc_top_preind_mc = copy.deepcopy(soc_top_preind)
soc_top_preind_mc.mask = dave_mask
soc_sub_preind_mc = copy.deepcopy(soc_sub_preind)
soc_sub_preind_mc.mask = dave_mask
soc_top_contemp_mc = copy.deepcopy(soc_top_contemp)
soc_top_contemp_mc.mask = dave_mask
soc_sub_contemp_mc = copy.deepcopy(soc_sub_contemp)
soc_sub_contemp_mc.mask = dave_mask


In [10]:
dave_mask

True

In [7]:
import time
# Print out McGuire's mask into NetCDF file
# Write into Netcdf to check the values
nc = NetCDFFile('dave_mask.nc','w',format='NETCDF4_CLASSIC')
# Define a set of dimensions to be used in the file
lat = nc.createDimension('lat', 360)
lon = nc.createDimension('lon', 720)
# Create variables
latitudes = nc.createVariable('lat', np.float32, ('lat',))
longitudes = nc.createVariable('lon', np.float32, ('lon',))
# Create the actual 4-d variable
d14c = nc.createVariable('Davemask', np.float32, ('lat','lon'))

# Add Global Attributes
nc.description = "Dave McGuire's mask."
nc.history = 'Created ' + time.ctime(time.time())
nc.source = 'netCDF4 python module'

# Add Variable Attributes
latitudes.units = 'degree_north'
longitudes.units = 'degree_east'
d14c.units = 'dimensionless'

# Put longitude and latitude
lats = np.arange(-89.75,90.25,0.5)
lons = np.arange(0.25,360.25,0.5)
levels = np.arange(1,11)
latitudes[:] = lats
longitudes[:] = lons
d14c[:,:] = dave_mask
nc.close()

In [ ]:
# Calculate total for comparison to Dave McGuire's paper
# Grid area
nlon = 720
nlat = 360
grid_area = np.ones((nlat, nlon))*-9999.
val1d_preind = np.ones((nlat, nlon))*float("nan")
val1d_contemp = np.ones((nlat, nlon))*float("nan")
val0d_preind = np.ones((nlat, nlon))*float("nan")
val0d_contemp = np.ones((nlat, nlon))*float("nan")
val_top_preind = np.ones((nlat, nlon))*float("nan")
val_sub_preind = np.ones((nlat, nlon))*float("nan")
val_top_contemp = np.ones((nlat, nlon))*float("nan")
val_sub_contemp = np.ones((nlat, nlon))*float("nan")

earth_area =  5.096e14;
lat = np.arange(-89.75, 90.25, 0.5)
res = 0.5;
for i in np.arange(0,nlat):
    for j in np.arange(0,nlon):
        grid_area[i,j] = (earth_area/2)*abs(np.sin((lat[i] - res/2)*np.pi/180) -
                    np.sin((lat[i] + res/2)*np.pi/180))/(360/res)
        val1d_preind[i,j] = soc_preind_mc[i,j] * grid_area[i,j]
        val1d_contemp[i,j] = soc_contemp_mc[i,j] * grid_area[i,j]
        val0d_preind[i,j] = soc0d_preind_mc[i,j] * grid_area[i,j]
        val0d_contemp[i,j] = soc0d_contemp_mc[i,j] * grid_area[i,j]
        val_top_preind[i,j] = soc_top_preind_mc[i,j] * grid_area[i,j]
        val_sub_preind[i,j] = soc_sub_preind_mc[i,j] * grid_area[i,j]
        val_top_contemp[i,j] = soc_top_contemp_mc[i,j] * grid_area[i,j]
        val_sub_contemp[i,j] = soc_sub_contemp_mc[i,j] * grid_area[i,j]



In [ ]:
plt.imshow(soc_contemp_mc)

In [ ]:
# Check or print results
val0d_preind[val0d_preind<0] = 0.
val0d_contemp[val0d_contemp<0] = 0.
val_top_preind[val_top_preind<0]= 0.
val_sub_preind[val_sub_preind<0]= 0.
val_top_contemp[val_top_contemp<0]= 0.
val_sub_contemp[val_sub_contemp<0]= 0.


total1d_preind = np.nansum(val1d_preind)/1e12
total1d_contemp = np.nansum(val1d_contemp)/1e12
total0d_preind = np.nansum(val0d_preind)/1e12
total0d_contemp = np.nansum(val0d_contemp)/1e12
total_top_preind = np.nansum(val_top_preind)/1e12
total_sub_preind = np.nansum(val_sub_preind)/1e12
total_top_contemp = np.nansum(val_top_contemp)/1e12
total_sub_contemp = np.nansum(val_sub_contemp)/1e12

print('ISAM-1D SOC stock 1960:')
print(total1d_preind)
print('ISAM-1D SOC stock 2009:')
print(total1d_contemp)
print('ISAM-0D SOC stock 1960:')
print(total0d_preind)
print('ISAM-0D SOC stock 2009:')
print(total0d_contemp)
print('=======================')
print('ISAM-1D topsoil SOC 1960:')
print(total_top_preind)
print('ISAM-1D topsoil SOC 2009:')
print(total_top_contemp)
print('ISAM-1D subsoil SOC 1960:')
print(total_sub_preind)
print('ISAM-1D subsoil SOC 2009:')
print(total_sub_contemp)

In [ ]:
235.47-233.6

In [ ]:
182*50

In [ ]:
182-37